In [143]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
import en_core_web_sm
import docx2txt
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from collections import Counter
from io import StringIO
import pdfplumber

filename = 'CV_MJD_2022.pdf'

if 'pdf' in filename:
    
    pdf = pdfplumber.open(filename) 
    page = pdf.pages[0]
    text = page.extract_text()
    
elif 'doc' in filename:

    # Passing docx file to process function
    text = docx2txt.process(filename)

jd = pd.read_excel("ability.xlsx")

In [144]:
jd_dict = jd.to_dict('list')

In [145]:
#Remove nan from the excel and nlp each of the words, store the value to a new 
new_dict = {}

for k, v in jd_dict.items():
    
    newlist = []
    for x in jd_dict[k]: 
        if str(x) != 'nan':
            x = nlp(x)
            newlist.append(x)
    
    new_dict[k] = newlist
                

In [146]:
matcher = PhraseMatcher(nlp.vocab)

for k, v in new_dict.items():
    
    matcher.add(k, None, *v)

In [147]:
text = str(text)
text = text.replace("\\n", "")
text = text.lower()
doc = nlp(text)

In [148]:
d = []
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    #print(span.text)
    d.append((rule_id, span.text))      

keywords = "\n".join(f'{i[0]} / {i[1]} ({j})' for i,j in Counter(d).items())

In [149]:
## convertimg string of keywords to dataframe
df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
df1 = pd.DataFrame(df.Keywords_List.str.split('/',1).tolist(),columns = ['Subject','Keyword'])
df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))

In [150]:
df3

,Subject,Keyword,Count
0,Project Management,project,1
1,Project Management,management,4
2,Project Management,stakeholders,4
3,Project Management,cost,1
4,Quality (Six Sigma),quality,2
5,Operation Management,process,2
6,Project Management,finance,2
7,Quality (Six Sigma),kpi,2
8,Project Management,risk,1
9,Operation Management,operations,4


In [152]:
name = filename.split('.')
name2 = name[0]
name2 = name2.lower()
## converting str to dataframe
name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)

dataf

,Candidate Name,Subject,Keyword,Count
0,cv_mjd_2022,Project Management,project,1
1,cv_mjd_2022,Project Management,management,4
2,cv_mjd_2022,Project Management,stakeholders,4
3,cv_mjd_2022,Project Management,cost,1
4,cv_mjd_2022,Quality (Six Sigma),quality,2
5,cv_mjd_2022,Operation Management,process,2
6,cv_mjd_2022,Project Management,finance,2
7,cv_mjd_2022,Quality (Six Sigma),kpi,2
8,cv_mjd_2022,Project Management,risk,1
9,cv_mjd_2022,Operation Management,operations,4
